In [2]:
import keras_ocr
import pytesseract

print(keras_ocr.__version__)
print(pytesseract.__version__)
# pytesseract는 tesseract-ocr에 종속적입니다. 아래 명령어를 통해 설치해야 할 수도 있습니다.
# !sudo apt install tesseract-ocr

0.8.8
0.3.8


# Step 1. 검증용 데이터셋 준비



In [3]:
image_urls = [
    'https://source.unsplash.com/M7mu6jXlcns/640x460',
    'https://source.unsplash.com/6jsp4iHc8hI/640x460',
    'https://source.unsplash.com/98uYQ-KupiE',
    'https://source.unsplash.com/j9JoYpaJH3A',
    'https://source.unsplash.com/eBkEJ9cH5b4',
    'https://iamabhik.files.wordpress.com/2011/04/32.jpg',
    'https://iamabhik.files.wordpress.com/2011/04/32.jpg',
    'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEusy9xTlwS0rm3IjDHgk11hTZplHAq0UcTRzdMEpiNkdRD5rx9VMVOqjNDowCRIScyG8&usqp=CAU',
    'https://opengraph.githubassets.com/9dbd9d4c5fda14473d2acd0d1fb8ce5347c59e1c6b9de893caca73cb25a171ab/OCR-D/assets',
    'https://ocr-d.de/assets/dfg_logo_eng.jpg',
    'https://m.media-amazon.com/images/I/41IZGtU2P1L._AC_.jpg'
]

images = [ keras_ocr.tools.read(url) for url in image_urls]
prediction_groups = [pipeline.recognize([url]) for url in image_urls]

NameError: name 'pipeline' is not defined

# Step 2. keras-ocr, Tesseract로 테스트 진행(Google OCR API는 선택 사항)


## keras-OCR

불러온 이미지 시각화해보기

In [ ]:
# Plot the predictions
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for idx, ax in enumerate(axs):
    keras_ocr.tools.drawAnnotations(image=images[idx], 
                                    predictions=prediction_groups[idx][0], ax=ax)

## Tesseract

Tesseract에서 사용하기 위해서 위에 URL로 가지고 와있는 image들을 cloud에 실제로 저장해주었습니다.

- 먼저 이미지의 데이터 형식을 파악해보았습니다.


In [ ]:
print(type(images[0]))

- 파이썬 이미지를 로컬에 저장하는 메소드들
https://www.delftstack.com/ko/howto/numpy/save-numpy-array-as-image/#cv2.imwrite%25ED%2595%25A8%25EC%2588%2598%25EB%25A5%25BC-%25EC%2582%25AC%25EC%259A%25A9%25ED%2595%2598%25EC%2597%25AC-numpy-%25EB%25B0%25B0%25EC%2597%25B4%25EC%259D%2584-%25EC%259D%25B4%25EB%25AF%25B8%25EC%25A7%2580%25EB%25A1%259C-%25EC%25A0%2580%25EC%259E%25A5


In [6]:
import imageio

for idx, f in enumerate(images):
    imageio.imwrite('image{}.jpeg'.format(idx), f)

In [9]:
import os
import pytesseract
from PIL import Image
from pytesseract import Output
import matplotlib.pyplot as plt

# OCR Engine modes(–oem):
# 0 - Legacy engine only.
# 1 - Neural nets LSTM engine only.
# 2 - Legacy + LSTM engines.
# 3 - Default, based on what is available.

# Page segmentation modes(–psm):
# 0 - Orientation and script detection (OSD) only.
# 1 - Automatic page segmentation with OSD.
# 2 - Automatic page segmentation, but no OSD, or OCR.
# 3 - Fully automatic page segmentation, but no OSD. (Default)
# 4 - Assume a single column of text of variable sizes.
# 5 - Assume a single uniform block of vertically aligned text.
# 6 - Assume a single uniform block of text.
# 7 - Treat the image as a single text line.
# 8 - Treat the image as a single word.
# 9 - Treat the image as a single word in a circle.
# 10 - Treat the image as a single character.
# 11 - Sparse text. Find as much text as possible in no particular order.
# 12 - Sparse text with OSD.
# 13 - Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.

def crop_word_regions(image_path='./images/sample.png', output_path='./output'):
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    custom_oem_psm_config = r'--oem 3 --psm 3'
    image = Image.open(image_path)

    recognized_data = pytesseract.image_to_data(
        image, lang='eng',    # 한국어라면 lang='kor'
        config=custom_oem_psm_config,
        output_type=Output.DICT
    )
    
    top_level = max(recognized_data['level'])
    index = 0
    cropped_image_path_list = []
    for i in range(len(recognized_data['level'])):
        level = recognized_data['level'][i]
    
        if level == top_level:
            left = recognized_data['left'][i]
            top = recognized_data['top'][i]
            width = recognized_data['width'][i]
            height = recognized_data['height'][i]
            
            output_img_path = os.path.join(output_path, f"{str(index).zfill(4)}.png")
            print(output_img_path)
            cropped_image = image.crop((
                left,
                top,
                left+width,
                top+height
            ))
            cropped_image.save(output_img_path)
            cropped_image_path_list.append(output_img_path)
            index += 1
    return cropped_image_path_list


work_dir = os.getenv('HOME')+'/aiffel/ocr_python'

for num in range(11):
    img_file_path = work_dir + '/image{}.jpeg'.format(num)   #테스트용 이미지 경로입니다. 본인이 선택한 파일명으로 바꿔주세요. 
    cropped_image_path_list = crop_word_regions(img_file_path, work_dir)

# cropped_image_path_list = crop_word_regions(img_file_path, work_dir)

/aiffel/aiffel/ocr_python/0000.png
/aiffel/aiffel/ocr_python/0000.png
/aiffel/aiffel/ocr_python/0001.png
/aiffel/aiffel/ocr_python/0002.png
/aiffel/aiffel/ocr_python/0003.png
/aiffel/aiffel/ocr_python/0004.png
/aiffel/aiffel/ocr_python/0005.png
/aiffel/aiffel/ocr_python/0006.png
/aiffel/aiffel/ocr_python/0007.png
/aiffel/aiffel/ocr_python/0000.png
/aiffel/aiffel/ocr_python/0001.png
/aiffel/aiffel/ocr_python/0002.png
/aiffel/aiffel/ocr_python/0003.png
/aiffel/aiffel/ocr_python/0004.png
/aiffel/aiffel/ocr_python/0005.png
/aiffel/aiffel/ocr_python/0006.png
/aiffel/aiffel/ocr_python/0007.png
/aiffel/aiffel/ocr_python/0008.png
/aiffel/aiffel/ocr_python/0009.png
/aiffel/aiffel/ocr_python/0010.png
/aiffel/aiffel/ocr_python/0011.png
/aiffel/aiffel/ocr_python/0012.png
/aiffel/aiffel/ocr_python/0013.png
/aiffel/aiffel/ocr_python/0014.png
/aiffel/aiffel/ocr_python/0015.png
/aiffel/aiffel/ocr_python/0016.png
/aiffel/aiffel/ocr_python/0017.png
/aiffel/aiffel/ocr_python/0018.png
/aiffel/aiffel/ocr_p

In [8]:
img_file_path

'/aiffel/aiffel/ocr_python/image10.jpeg'

In [ ]:
import matplotlib.pyplot as plt
import keras_ocr

# keras-ocr이 detector과 recognizer를 위한 모델을 자동으로 다운로드받게 됩니다. 
pipeline = keras_ocr.pipeline.Pipeline()